## in this script we're aggregating cases from zipcode into PUMA and then into KC region to be prepped for MASCOT GLM

In [ ]:
import pandas as pd
import xlrd
import numpy as np

In [ ]:
kc_data = 'kc_cases.csv' #data taken from https://kingcounty.gov/depts/health/covid-19/data/daily-summary.aspx

In [ ]:
dfnew = pd.read_csv(kc_data, parse_dates = [2])
dfnew['Positives'] = dfnew['Positives'].astype("str").astype('float')

#dfnew['geo_id'] = dfnew['geo_id'].astype('int')

In [ ]:
# go from zipcode to PUMA
dfnew['PUMA'] = np.nan
for index, rows in dfnew.iterrows():
    if rows.geo_id in (98103, 98107, 98117):
        dfnew.loc[index, 'PUMA'] = "11601" 
    elif rows.geo_id in (98105, 98115, 98125, 98195):
        dfnew.loc[index, 'PUMA'] = "11602" 
    elif rows.geo_id in (98101, 98102, 98104, 98109, 98119, 98121, 98154, 98164, 98199):
        dfnew.loc[index, 'PUMA'] = "11603" 
    elif rows.geo_id in (98112, 98118, 98122, 98144):
        dfnew.loc[index, 'PUMA'] = "11604" 
    elif rows.geo_id in (98106, 98108, 98116, 98126, 98134, 98136):
        dfnew.loc[index, 'PUMA'] = "11605" 
    elif rows.geo_id in (98133, 98155, 98177, 98011, 98028):
        dfnew.loc[index, 'PUMA'] = "11606" 
    elif rows.geo_id in (98033, 98034, 98052):
        dfnew.loc[index, 'PUMA'] = "11607" 
    elif rows.geo_id in (98004, 98005, 98006, 98007, 98008, 98039):
        dfnew.loc[index, 'PUMA'] = "11608" 
    elif rows.geo_id in (98040, 98029, 98074, 98075):
        dfnew.loc[index, 'PUMA'] = "11609" 
    elif rows.geo_id in (98055, 98057, 98056, 98178):
        dfnew.loc[index, 'PUMA'] = "11610" 
    elif rows.geo_id in (98146, 98148, 98166, 98168, 98188):
        dfnew.loc[index, 'PUMA'] = "11611" 
    elif rows.geo_id in (98003, 98023, 98198, 98070):
        dfnew.loc[index, 'PUMA'] = "11612" 
    elif rows.geo_id in (98030, 98031, 98032, 98092):
        dfnew.loc[index, 'PUMA'] = "11613" 
    elif rows.geo_id in (98001, 98002, 98047):
        dfnew.loc[index, 'PUMA'] = "11614" 
    elif rows.geo_id in (98010, 98022, 98038, 98051, 98027, 98042, 98059, 98058):
        dfnew.loc[index, 'PUMA'] = "11615" 
    elif rows.geo_id in (98045, 98065, 98014, 98077, 98053, 98024, 98072, 98019):
        dfnew.loc[index, 'PUMA'] = "11616" 

In [ ]:
#go from PUMA to KC region
north_kc = ["11601", "11602", "11603", "11604", "11605", "11606", "11607", "11608", "11609",  "11616" ]
south_kc = [ "11610", "11611", "11612", "11613", "11614", "11615"]
dfnew['region'] = np.nan
for index, row in dfnew.iterrows():
    if row.PUMA in north_kc:
        dfnew.loc[index, "region"] = "nKC"
    elif row.PUMA in south_kc:
        dfnew.loc[index, "region"] = "sKC"
    

In [ ]:
#14 rolling average day average for hospitalization by region
dfnew['Date'] = pd.to_datetime(dfnew['date']) 
df_test = dfnew.groupby(['region', pd.Grouper(key='Date')])['Positives'].sum().rolling(14, min_periods=1).mean()
df = df_test.reset_index()
df


In [ ]:
#df.to_csv("../data-files/cases_per_100_by_region.csv")

In [ ]:
#MASCOT will log normalize these values and thus we can't have any 0 values. 
df.Positives[df.Positives <= 0] = 0.1

In [ ]:
nkc_df = df[df['region'] == "nKC"]
skc_df = df[df['region'] == "sKC"]

skc_df = skc_df[['region', 'Date', 'Positives']]
nkc_df = nkc_df[['region', 'Date', 'Positives']]

skc_df = skc_df.rename(columns = {"Positives": "Positives_skc"}).reset_index()
nkc_df = nkc_df.rename(columns = {"Positives": "Positives_nkc"}).reset_index()

In [ ]:
#merging n and s datasets together
merged_df = skc_df.merge(nkc_df, on = "Date")

In [ ]:
merged_df

In [ ]:
#setting everything up for back filling some dates where we don't have case data for since MASCOT doesnt like NA or 0s
mar6_df = merged_df[merged_df["Date"] <= "2022-03-06"]
mar6_df.index = mar6_df.Date
first_date = pd.Timestamp('2019-12-21')
end_date = pd.Timestamp('2022-03-05')
dates = pd.date_range(first_date, end_date, freq='D')
dates.name = 'date'

In [ ]:
mar6_df = mar6_df.reindex(dates,  fill_value = 0.1)

In [ ]:
df_np = mar6_df[['Positives_skc', 'Positives_nkc']].to_numpy()

In [ ]:
#in order to fit the matrix format of MASCOT GLM, we need it to go SKC, NKC in backwards in time order
blah = df_np.tolist()
blah.reverse()
len(blah)

In [ ]:
#this prints out the predictors that can be copy and pasted into the template
print(*blah, sep=' ')
